In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_03 = pd.read_csv(folderpath + 'data_op_03.csv')
data_op_03.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
1,1,2,0.0015,0.0003,100.0,1584.55,1403.96,14.62,2388.01,521.81,2388.15,0.03,3,1
7,1,8,0.0011,0.0000,100.0,1589.55,1407.59,14.62,2388.10,521.74,2388.03,0.03,3,1
8,1,9,0.0011,0.0020,100.0,1586.18,1399.01,14.62,2388.10,521.72,2388.08,0.03,3,1
16,1,17,0.0003,0.0000,100.0,1584.57,1410.11,14.62,2388.06,522.17,2388.10,0.03,3,1
19,1,20,0.0004,0.0017,100.0,1581.33,1399.44,14.62,2388.09,521.62,2388.06,0.03,3,1


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_03 = tf_remove_sensor_15.fit_transform(data_op_03)
data_op_03.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
1,1,2,0.0015,0.0003,100.0,1584.55,1403.96,14.62,2388.01,521.81,2388.15,3,1
7,1,8,0.0011,0.0000,100.0,1589.55,1407.59,14.62,2388.10,521.74,2388.03,3,1
8,1,9,0.0011,0.0020,100.0,1586.18,1399.01,14.62,2388.10,521.72,2388.08,3,1
16,1,17,0.0003,0.0000,100.0,1584.57,1410.11,14.62,2388.06,522.17,2388.10,3,1
19,1,20,0.0004,0.0017,100.0,1581.33,1399.44,14.62,2388.09,521.62,2388.06,3,1


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_03)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
1,1584.55,1403.96,14.62,2388.01,521.81,2388.15,1
7,1589.55,1407.59,14.62,2388.10,521.74,2388.03,1
8,1586.18,1399.01,14.62,2388.10,521.72,2388.08,1
16,1584.57,1410.11,14.62,2388.06,522.17,2388.10,1
19,1581.33,1399.44,14.62,2388.09,521.62,2388.06,1


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-1.040896  , -0.62635123, -1.        , -1.37994051,  0.63774241,
         0.65278365],
       [-0.19248051, -0.21074616, -1.        , -0.05996073,  0.5391007 ,
        -1.09204307],
       [-0.76431255, -1.19308541, -1.        , -0.05996073,  0.51091736,
        -0.36503194],
       ...,
       [ 0.79168145,  0.75556424, -1.        ,  1.55334789, -1.33509174,
         2.39761038],
       [ 1.7639656 ,  1.08530049, -1.        ,  2.4333344 , -2.16650042,
         2.97921928],
       [ 1.69609236,  1.87872835, -1.        ,  2.4333344 , -1.67329188,
         1.81600147]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

In [7]:
from phm08ds.models import experiment

## Load best mlp from random search

In [8]:
from sklearn.externals import joblib

random_search_mlp = joblib.load('Results/old_models/clf_mlp.pkl')
random_search_svm = joblib.load('Results/old_models/clf_svm.pkl')

In [9]:
clf_mlp = random_search_mlp['MLP'].best_estimator_
clf_svm_linear = random_search_svm['SVM-Linear'].best_estimator_
clf_svm_rbf = random_search_svm['SVM-RBF'].best_estimator_
clf_svm_poly = random_search_svm['SVM-Poly'].best_estimator_
clf_svm_sigmoid = random_search_svm['SVM-Sigmoid'].best_estimator_

In [10]:
# random_search_svm

## Put all clf in a dictionary:

In [11]:
classifiers = {'MLP': clf_mlp, 
               'SVM-Linear': clf_svm_linear,
               'SVM-RBF': clf_svm_rbf,
               'SVM-Poly': clf_svm_poly,
               'SVM-Sigmoid': clf_svm_sigmoid}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

In [12]:
kfolds = 10
clf_outputs = experiment.run_classifiers(data_with_features_std, labels, classifiers, kfolds)

## Performance assessment

Saving variables in a dictionary:

In [13]:
results = {}
results['train'] = experiment.results_clf(4, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = experiment.results_clf(4, clf_outputs['test']['true'], clf_outputs['test']['pred'])

/home/navar/anaconda3/envs/phm08ds/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/envs/phm08ds/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/envs/phm08ds/lib/python3.6/site-packages/phm08ds/models/experiment.py:149: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)


## Savel results, models and pipeline to a .pkl file 

In [14]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [15]:
experiment.save_models(classifiers)
experiment.save_pipeline(data_preprocessing)

## Save results to CSVs and figures

In [16]:
experiment.export_results(results['test'], 'test')
experiment.export_results(results['train'], 'train')

/mnt/Work/Mestrado/Mestrado/MP-Safety_ITA/Analyses_and_Experiments/PHM08_data_science/notebooks/E03_PHM08-train-best_MLP_and_SVM/model_selection-OP_03-navarmn
MLP
SVM-Linear
SVM-RBF
SVM-Poly
SVM-Sigmoid
/mnt/Work/Mestrado/Mestrado/MP-Safety_ITA/Analyses_and_Experiments/PHM08_data_science/notebooks/E03_PHM08-train-best_MLP_and_SVM/model_selection-OP_03-navarmn
MLP
SVM-Linear
SVM-RBF
SVM-Poly
SVM-Sigmoid


## Savel results, models and pipeline to a .pkl file 

In [17]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [18]:
experiment.save_models(clf_mlp, name='clf_mlp')
experiment.save_pipeline(data_preprocessing)